In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import pandas as pd
import os


In [2]:
# Dataset paths
train_dir = "/kaggle/input/fetal-brain-abnormalities/train"
valid_dir = "/kaggle/input/fetal-brain-abnormalities/valid"
test_dir = "/kaggle/input/fetal-brain-abnormalities/test"

In [3]:
train_csv_path = os.path.join(train_dir, "_classes.csv")
valid_csv_path = os.path.join(valid_dir, "_classes.csv")
test_csv_path = os.path.join(test_dir, "_classes.csv")

In [6]:

# Load CSV files
train_df = pd.read_csv(train_csv_path)
valid_df = pd.read_csv(valid_csv_path)
test_df = pd.read_csv(test_csv_path)

In [7]:
# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [8]:
# Extract class labels (excluding filename column)
class_labels = train_df.columns[1:].tolist()

In [9]:
def create_datagen(df, img_dir):
    datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=20)
    
    def generator():
        for _, row in df.iterrows():
            img_path = os.path.join(img_dir, row['filename'])
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            label = row[class_labels].values.astype(float)
            yield img_array, label
    
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(len(class_labels),), dtype=tf.float32)
        )
    )
    return dataset.batch(BATCH_SIZE)

In [10]:
train_dataset = create_datagen(train_df, train_dir)
valid_dataset = create_datagen(valid_df, valid_dir)
test_dataset = create_datagen(test_df, test_dir)

In [11]:
# Load pre-trained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model weights


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
# Custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
out = Dense(len(class_labels), activation='sigmoid')(x)  # Sigmoid for multi-label classification

In [13]:
model = Model(inputs=base_model.input, outputs=out)

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
# Train the model
model.fit(train_dataset, validation_data=valid_dataset, epochs=10)

Epoch 1/10
     45/Unknown 24s 309ms/step - accuracy: 0.1219 - loss: 0.3120

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 457ms/step - accuracy: 0.1226 - loss: 0.3105 - val_accuracy: 0.1839 - val_loss: 0.2068
Epoch 2/10
 1/45 ━━━━━━━━━━━━━━━━━━━━ 7s 178ms/step - accuracy: 0.1562 - loss: 0.2004

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.1638 - loss: 0.2122

...

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.2145 - loss: 0.2053

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 123ms/step - accuracy: 0.2147 - loss: 0.2053 - val_accuracy: 0.2586 - val_loss: 0.1994


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [16]:

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 658ms/step - accuracy: 0.2398 - loss: 0.2047
Test Accuracy: 21.59%


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
